In [1]:
import pandas as pd
import os

In [5]:
# Sous-dossiers contenant les fichiers CSV
carraud_dir = "CARRAUD"
daudet_dir = "DAUDET"

In [ ]:
# Fonction pour vérifier l'existence du répertoire et lire les fichiers CSV
def read_csv_files(directory):
    if not os.path.exists(directory):
        print(f"Le répertoire {directory} n'existe pas.")
        return []
    
    dataframes = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath, sep=";")
            dataframes.append(df)
    return dataframes

# Combiner les données de tous les DataFrames en un seul
def combine_dataframes(dataframes):
    if not dataframes:
        return pd.DataFrame()  # Retourne un DataFrame vide si la liste est vide
    
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

# Lire les fichiers et combiner les données
carraud_dfs = read_csv_files(carraud_dir)
daudet_dfs = read_csv_files(daudet_dir)
combined_df = combine_dataframes(carraud_dfs + daudet_dfs)

if combined_df.empty:
    print("Aucun fichier CSV trouvé dans les répertoires spécifiés.")
else:
    # Ajouter les colonnes nécessaires
    combined_df['périmètre'] = combined_df.apply(lambda row: 'X' if any(pd.notna(row[col]) for col in combined_df.columns if col != 'Token') else '', axis=1)

    def process_annotations(row):
        annotations = [row[col] for col in combined_df.columns if col != 'Token' and col != 'périmètre']
        annotations = [ann for ann in annotations if pd.notna(ann)]
        if len(annotations) >= 4:
            # Vérifier la majorité
            most_common = max(set(annotations), key=annotations.count)
            if annotations.count(most_common) >= 4:
                row['accord'] = most_common
                return row
        if len(annotations) > 0:
            row['désaccord'] = ','.join(set(annotations))
            row['confusion'] = 'X'
        return row

    combined_df = combined_df.apply(process_annotations, axis=1)
    combined_df['accord'] = combined_df.get('accord', "")
    combined_df['désaccord'] = combined_df.get('désaccord', "")
    combined_df['confusion'] = combined_df.get('confusion', "")

    # Sauvegarder le résultat dans un nouveau fichier CSV
    output_filepath = "resultat_annotations.csv"
    combined_df.to_csv(output_filepath, sep=";", index=False)
    print(f"Le fichier résultant a été sauvegardé à {output_filepath}")

C:\Users\user\AppData\Local\Temp\ipykernel_4928\2108675908.py:11: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, sep=";")
C:\Users\user\AppData\Local\Temp\ipykernel_4928\2108675908.py:11: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, sep=";")
